In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np

from firedrake.petsc import PETSc

In [ ]:

import os
current_path = os.getcwd()
print(current_path)
my_io_path = ''

print(my_io_path)



## Multi-phase flow by level-set method.

In [ ]:
from firedrake.pyplot import *
n = 20  # per unit length
mesh = RectangleMesh(n, 2*n, 1, 2)
fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend(loc='upper left')

In [ ]:
# Function spaces
V = VectorFunctionSpace(mesh, 'P', 2)   # velocity
Q = FunctionSpace(mesh, 'P', 1)         # pressure
W = MixedFunctionSpace([V, Q])
Z = FunctionSpace(mesh, 'P', 1)         # level-set

# Data and boundary conditions
mu = Constant(0.1)
g = Constant((0.0,-1.0))
rho1 = 10.0
rho2 = 1.e-4
T = 4.0
dt = 1.e-2

bcNS_tb = DirichletBC(W.sub(0), Constant((0.0, 0.0)), (3,4))
bcNS_lr = DirichletBC(W.sub(0).sub(0), Constant(0.0), (1,2))

bcsNS = (bcNS_tb, bcNS_lr)

nullsp = MixedVectorSpaceBasis(W, [W.sub(0), VectorSpaceBasis(constant=True)])

### Variational problems

In [ ]:
def time_step_NS(u, v, p, q, rhoh, mu, g, dt, u_old):
    # u,p   :   TrialFunctions
    # v,q   :   TestFunctions
    # rhoh      density Function
    # u_old :   old velocity Function

    a = rhoh / dt * inner(u, v) * dx \
        + 2 * mu * inner(sym(grad(u)), sym(grad(v))) * dx  \
        + rhoh * inner(dot(grad(u), u_old), v) * dx  \
        - div(v) * p * dx  \
        + q * div(u) * dx
    L = rhoh / dt * inner(u_old, v) * dx \
        + rhoh * inner(g, v) * dx

    return a, L

def time_step_level_set(phi, psi, uh, dt, phi_old):
    # phi       :   TrialFunction
    # psi       :   TestFunction
    # phi_old   :   old level-set Function
    # uh        :   advecting velocity Function


    a = 1.0 / dt * phi * psi * dx  \
        + inner(grad(phi), uh) * psi * dx
    L = 1.0 / dt * phi_old * psi * dx

    return a, L

### Initialization and post-processing setup.

In [ ]:
# Initialization
wh = Function(W)
uh, ph = wh.subfunctions

R = 0.25
phih = Function(Z)
x = SpatialCoordinate(mesh)
phih.interpolate( sqrt( (x[0]-0.5)**2 + (x[1]-0.5)**2 ) - R )

rhoh = Function(Z)
rhoh.interpolate(conditional(gt(phih, 0.0), rho1, rho2))

# Plot of initial density
fig, ax = plt.subplots()
col = tripcolor(rhoh, axes=ax)
plt.colorbar(col)
plt.title('rho')

# vtk output for Paraview
basename = 'LS'
outfileU = File(my_io_path+"output/"+basename+"velocity.pvd")
outfileP = File(my_io_path+"output/"+basename+"pressure.pvd")
outfilePhi = File(my_io_path+"output/"+basename+"phi.pvd")
uh.rename("Velocity")   
ph.rename("Pressure")  
phih.rename("Phi")   
outfileU.write(uh)
outfileP.write(ph)
outfilePhi.write(phih)

### FE functions: trial, test, old solution

In [ ]:
u, p = TrialFunctions(W)
v, q = TestFunctions(W)
u_old = Function(V)
u_old.assign(uh)

phi = TrialFunction(Z)
psi = TestFunction(Z)
phi_old = Function(Z)
phi_old.assign(phih)

### Time loop

In [ ]:
t_vec = np.arange(0, T+0.1*dt, dt)  # T+0.1*dt to include also T: range/arange exclude the upper bound of the range
mass = [0.0] * (len(t_vec))
mass[0] = assemble(rhoh*dx)


for ii in range(1, len(t_vec)):     # start from 1 to skip t=0
    t = t_vec[ii]

    a_NS, L_NS = time_step_NS(u, v, p, q, rhoh, mu, g, dt, u_old)
    pb_NS = LinearVariationalProblem(a_NS, L_NS, wh, bcsNS)
    solver_NS = LinearVariationalSolver(pb_NS, nullspace = nullsp)
    solver_NS.solve()
    uh, ph = wh.subfunctions

    a_LS, L_LS = time_step_level_set(phi, psi, uh, dt, phi_old)
    pb_LS = LinearVariationalProblem(a_LS, L_LS, phih)
    solver_LS = LinearVariationalSolver(pb_LS)
    solver_LS.solve()


    if (ii % 10 == 0):
        uh.rename("Velocity")
        ph.rename("Pressure")
        phih.rename("Phi")
        outfileU.write(uh, time=t)
        outfileP.write(ph, time=t)
        outfilePhi.write(phih, time=t)

    # Update solutions and parameters for next time step
    u_old.assign(uh)
    phi_old.assign(phih)
    #Reinterpolate rho
    rhoh.interpolate(conditional(gt(phih, 0.0), rho1, rho2))

    mass[ii] = assemble(rhoh*dx)

    if (ii % 10 == 0):
        print("Time = ", t, '      mass =', mass[ii])



In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

fig, ax = plt.subplots()
col = tripcolor(phih, axes=ax)
plt.colorbar(col)
plt.title('phi')
fig, ax = plt.subplots()
col = tripcolor(rhoh, axes=ax)
plt.colorbar(col)
plt.title('rho')

fig, ax1 = plt.subplots()
line1 = ax1.plot(t_vec,
                 [float('nan') if x==0 else x for x in mass],
                    # replace 0 with nan to prevent plotting of 0 values
                    # -> useful if execution is interrupted before filling the whole mass vector
                 label='mass')
line2 = ax1.plot(t_vec,
                 [float('nan') if x==0 else mass[0] for x in mass],
                    # replace 0 with nan to prevent plotting of 0 values
                    # -> useful if execution is interrupted before filling the whole mass vector
                 '--',
                 label='initial value')